# **Loading Libraries**

In [1]:
# Package Installation
!pip install transformers[torch] -q
!pip install accelerate -U -q
!pip install emoji -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.5 MB/s eta 0:00:00


In [2]:
import re
import string

import numpy as np
import torch
import pandas as pd
import emoji

from transformers import Trainer, TrainingArguments
from transformers import pipeline
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

# **Data Preparation**

In [4]:
# Dataset is loaded into a dataframe
df = pd.read_csv("/content/train_refined.csv")
test_df = pd.read_csv("/content/test_data.csv")

#Information about the structure of both train and test dataset is printed
df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4265 entries, 0 to 4264
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   campaign_id          4265 non-null   object
 1   comment_id           4265 non-null   int64 
 2   comment_description  4265 non-null   object
 3   sentiment            4265 non-null   object
dtypes: int64(1), object(3)
memory usage: 133.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   campaign_id          1187 non-null   int64 
 1   comment_id           1187 non-null   int64 
 2   comment_description  1187 non-null   object
dtypes: int64(2), object(1)
memory usage: 27.9+ KB


# **Preprocessing**

### **Remove Empty Tweets**

In [5]:
# blank tweets (tweets with no comment_description) are identified
blank_tweets = df[df['comment_description'].isnull() | (df['comment_description'] == '')]

# The rows and indices of the blank tweets are printed
# This provides insight into the location and content of blank tweets within the dataset.
for index, row in blank_tweets.iterrows():
    print("Index:", index)
    print("Row:", row)
    print()

These blank tweets are removed here. We remove rows with empty values and empty string in the 'comment_description' column. Afterwards the
index is reset.

In [6]:
# Rows with empty or missing values in the 'comment_description' column are removed
# This step ensures that only tweets with valid descriptions are retained in the dataset.
df = df.dropna(subset=['comment_description'])
df = df[df['comment_description'] != '']

# The index is reset to maintain continuity after removing rows
df = df.reset_index(drop=True)
print(df.count())

campaign_id            4265
comment_id             4265
comment_description    4265
sentiment              4265
dtype: int64


### **Remove Duplicate Tweets**

This function helps us in finding duplicate tweets in the training set.
It shortens each tweets to its first 100 words. If tweet is in string format,
then it returns first 100 chracters otherwise returns NaN.

In [7]:
def duplicate_tweets(tweet):
    if isinstance(tweet, str):
        shortened_tweet = tweet[:100]
    else:
        shortened_tweet = np.nan
    return shortened_tweet

# First 100 characters of each row of comment_description are saved in a new column
df['first100charactersoftweets'] = df['comment_description'].apply(duplicate_tweets)

# Duplicates are dropped based on the first 100 characters and newly created column is deleted
df = df.drop_duplicates(subset='first100charactersoftweets', keep="first")
df = df.drop(columns=['first100charactersoftweets'])

print(df.count())

campaign_id            4265
comment_id             4265
comment_description    4265
sentiment              4265
dtype: int64


# **Removing URL, Handles, Punctuation**

In this section, we do the basic preprocessing of the individual tweets.
URLs, mentions, handles, punctuation are removed from the tweet text.
This step is essential for cleaning the text data before further analysis or modeling.

In [8]:
def preprocess_tweet(text):
    text = emoji.demojize(text)
    text = re.sub(r'https?://\S+', '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text) # Remove mentions/handles
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    text = text.lower() #Lowercase the text
    return text

In [9]:
#Visualization of dataset before preprocessing
df.head(10)

,campaign_id,comment_id,comment_description,sentiment
0,2212,17908351952371091,لخسارة الوزن الزائد والكرش بمدة قياسية مع عدم ...,Positive
1,2217,17935944230085744,🔥🔥🔥,Positive
2,2215S,17899518356507020,This is so good😍 would be great it If you add ...,Positive
3,2214,18014766136389857,😍,Positive
4,2203,17924318627206870,طبق رائع ومميز تبارك الرحمن تسلم ايدك يارب 😍,Positive
5,2217,18032092822349781,@zainab.aleqabi مفعل هذا التطبيق بالعراق ؟,Neutral
6,2215S,18122714995274518,@muhamyat_alfreej,Neutral
7,2215S,17947316074891383,ابدعتي ام شريف,Positive
8,2215C,18201796876154595,شكلو و قوامو روعة سلم ايديكي❤️❤️,Positive
9,2211,17955037840709480,❤️❤️❤️❤️❤️❤️❤️,Positive


In [10]:
df['sentiment'].value_counts()

sentiment
Positive    3402
Neutral      698
Negative     165
Name: count, dtype: int64

In [11]:
# preprocess_tweet function is applied to each row of the dataframe
# This ensures that each tweet's text is cleaned and ready for further processing.
df['comment_description'] = df['comment_description'].apply(preprocess_tweet)

In [12]:
df.head(10)

,campaign_id,comment_id,comment_description,sentiment
0,2212,17908351952371091,لخسارة الوزن الزائد والكرش بمدة قياسية مع عدم ...,Positive
1,2217,17935944230085744,firefirefire,Positive
2,2215S,17899518356507020,this is so goodsmilingfacewithhearteyes would ...,Positive
3,2214,18014766136389857,smilingfacewithhearteyes,Positive
4,2203,17924318627206870,طبق رائع ومميز تبارك الرحمن تسلم ايدك يارب smi...,Positive
5,2217,18032092822349781,aleqabi مفعل هذا التطبيق بالعراق ؟,Neutral
6,2215S,18122714995274518,,Neutral
7,2215S,17947316074891383,ابدعتي ام شريف,Positive
8,2215C,18201796876154595,شكلو و قوامو روعة سلم ايديكيredheartredheart,Positive
9,2211,17955037840709480,redheartredheartredheartredheartredheartredhea...,Positive


# **Language Detection and Translation**

In this cell, I have provided some different efficient approaches to tackle this task which can help improve the model in future.
1. We can detect the language of input text using **langdetect** library. If the detected language is Arabic, we translate it to english using **Google Translate API** or **googletrans** library. This way, all the elements of dataset will be in one language and we can train BERT model instead of mBERT.
2. We can make customized preprocessing based on the detected langauge. AraBERT tokenizer can be used for Arabic text instead of mBERT tokenizer.

# **Model Fine-tuning**

In [13]:
# Dataset is split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2)

# The count of samples in the training set is printed.
print("Training Set Count:")
print(train_df.count())

# The count of samples in the validation set is printed.
print("\n\nValidation Set Count:")
print(val_df.count())

Training Set Count:
campaign_id            3412
comment_id             3412
comment_description    3412
sentiment              3412
dtype: int64


Validation Set Count:
campaign_id            853
comment_id             853
comment_description    853
sentiment              853
dtype: int64


In [14]:
# Custom dataset class for sentiment analysis using BERT tokenizer.
class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        text = str(self.data.comment_description.iloc[index])
        # The text is tokenized using BERT tokenizer
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )

        label = self.data.sentiment.iloc[index]

        if label == 'Positive':
          sentiment_id = 2
        elif label == 'Neutral':
          sentiment_id = 1
        else:
          sentiment_id = 0

        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(sentiment_id, dtype=torch.long)
            #'targets': torch.tensor(self.data.sentiment.iloc[index])
        }

    def __len__(self):
        return self.len



In [15]:
# DataFrame is converted to Dataset for training and validation
train_dataset = SentimentDataset(train_df, tokenizer)
val_dataset = SentimentDataset(val_df, tokenizer)

# Lengths of the training and validation datasets are printed
print(len(train_dataset))
print(len(val_dataset))

3412
853


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir = '/results',          # output directory
    num_train_epochs = 10,              # total number of training epochs
    per_device_train_batch_size = 32,  # batch size per device during training
    per_device_eval_batch_size = 32,   # batch size for evaluation
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = '/logs',            # directory for storing logs
    logging_steps = 10,
    #evaluation_strategy = 'epoch'
)

# Trainer object for training the model
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset
)

# Model is trained
trainer.train()

Step,Training Loss
10,0.790700
20,0.516600
30,0.524900
40,0.503300
50,0.524000
60,0.464700
70,0.487100
80,0.383800
90,0.405800
100,0.442500


Step,Training Loss
10,0.790700
20,0.516600
30,0.524900
40,0.503300
50,0.524000
60,0.464700
70,0.487100
80,0.383800
90,0.405800
100,0.442500


TrainOutput(global_step=1070, training_loss=0.14017971263258397, metrics={'train_runtime': 1538.2844, 'train_samples_per_second': 22.181, 'train_steps_per_second': 0.696, 'total_flos': 4488714906439680.0, 'train_loss': 0.14017971263258397, 'epoch': 10.0})

# **Model Saving**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Fine-tuned model and tokenizer are saved to the specified directory
model_path = "/content/drive/MyDrive/Sila"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Rewaa/tokenizer_config.json',
 '/content/drive/MyDrive/Rewaa/special_tokens_map.json',
 '/content/drive/MyDrive/Rewaa/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Rewaa/added_tokens.json',
 '/content/drive/MyDrive/Rewaa/tokenizer.json')

# **Inference**

In [ ]:
# The saved fine-tuned model is loaded
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [16]:
# A mapping is defined from sentiment labels to human-readable names
SENTIMENT_LABELS = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

def predict_sentiment(text):
    # The input text is tokenized
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        return_token_type_ids=True,
        return_attention_mask=True,
        truncation=True
    )

    # Input is converted to PyTorch tensors
    input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long).unsqueeze(0)
    attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long).unsqueeze(0)

    # Forward pass through the model
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predicted_label_id = torch.argmax(outputs.logits).item()

    # Predicted label ID is mapped to sentiment class label
    predicted_sentiment = SENTIMENT_LABELS[predicted_label_id]
    return predicted_sentiment

In [17]:
# Sentiment of a new text is predicted here
input_text = "You are very bad"
preprocessed_text = preprocess_tweet(input_text)
predicted_sentiment = predict_sentiment(preprocessed_text)
print("Predicted Sentiment:", predicted_sentiment)

Predicted Sentiment: Negative


# **Bulk-Inference**

In [19]:
test_df

,campaign_id,comment_id,comment_description
0,2333,17997297287137596,👏👏👏👏
1,2333,18070158229448348,🤍
2,2333,18380170993069042,😍❤️
3,2333,18034499602651743,😮❤️😍❤️❤️
4,2333,17978639144449496,🔥🔥🤟
...,...,...,...
1182,2333,17863240096059832,Oh cool. It's amazing how they keep doing some...
1183,2333,17917621504221036,@nishaldesai31 hehe sorry that’s not for sale😜
1184,2333,17921662243202909,Give us the ♥️ thats placed behind😍
1185,2333,18019013590772545,🧿🧿💍🤞


In [20]:
test_df['sentiment'] = test_df['comment_description'].apply(predict_sentiment)

In [23]:
test_df.to_csv('test_refined.csv', index=False)

In [22]:
test_df

,campaign_id,comment_id,comment_description,sentiment
0,2333,17997297287137596,👏👏👏👏,Positive
1,2333,18070158229448348,🤍,Neutral
2,2333,18380170993069042,😍❤️,Positive
3,2333,18034499602651743,😮❤️😍❤️❤️,Positive
4,2333,17978639144449496,🔥🔥🤟,Positive
...,...,...,...,...
1182,2333,17863240096059832,Oh cool. It's amazing how they keep doing some...,Positive
1183,2333,17917621504221036,@nishaldesai31 hehe sorry that’s not for sale😜,Neutral
1184,2333,17921662243202909,Give us the ♥️ thats placed behind😍,Positive
1185,2333,18019013590772545,🧿🧿💍🤞,Neutral
